In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.metrics import mean_absolute_error
import lightgbm as lgb
from lightgbm import LGBMRegressor
import warnings
warnings.filterwarnings('ignore')

# 1. Data Cleaning

In [ ]:
df = pd.read_csv('MTA_Daily_Ridership_Data.csv')
df.head()

In [ ]:
columns = [
    'Subways: Total Estimated Ridership',
    'Buses: Total Estimated Ridership',
    'LIRR: Total Estimated Ridership',
    'Metro-North: Total Estimated Ridership',
    'Staten Island Railway: Total Estimated Ridership',
]

In [ ]:
# Filter DataFrame in place to keep only the specified columns plus Date
df = df[['Date'] + columns]
df.head()

In [ ]:
df['Date'] = pd.to_datetime(df['Date'], format="%m/%d/%Y")

**Adding Temporal Features**

In [ ]:
df['Day of Week'] = df['Date'].dt.dayofweek  # 0=Monday, 6=Sunday
df['Month'] = df['Date'].dt.month
df['Year'] = df['Date'].dt.year
df['Is Weekend'] = df['Day of Week'].isin([5,6]).astype(int)
df.head()

In [ ]:
import holidays

us_holidays = holidays.US()
# Convert datetime index to date objects for proper comparison with holidays
df['Is Holiday'] = [1 if date in us_holidays else 0 for date in df['Date'].dt.date]

**Add External Event Features**: COVID, Hurricanes/Floods, Concerts and Sports events

In [ ]:
covid_start = pd.to_datetime('2020-03-15')
covid_end = pd.to_datetime('2020-06-30')
df['COVID'] = ((df['Date'] >= covid_start) & (df['Date'] <= covid_end)).astype(int)


In [ ]:
NY_HURRICANE_DATES = pd.Series([
    '2020-07-10',
    '2020-07-11', # Tropical Storm Fay 
    '2020-08-04', # Hurricane Isaias
    '2020-08-31', # Hurricane Laura
    '2020-09-21', # Hurricane Teddy
    '2020-09-22',
    '2021-07-09', # Hurricane Elsa
    '2021-08-18', # Tropical Storm Fred
    '2021-08-22', # Hurricane Henri
    '2021-09-01', # Hurricane Ida
    '2022-10-01', # Hurricane Nicole
    '2022-10-02', # Hurricane Nicole
    '2022-10-03', # Hurricane Nicole
    '2022-10-04', # Hurricane Nicole
    '2023-09-28', # Tropical Storm Ophelia
    '2023-09-29',
    '2020-09-30',
    '2024-08-07', # Hurricane Beryl
    '2024-08-08', # Hurricane Beryl
    '2024-08-09', # Hurricane Beryl
    '2025-07-07', # Tropical Storm Chantal
    '2025-07-08', # Tropical Storm Chantal
])

In [ ]:
NY_HURRICANE_DATES = pd.to_datetime(NY_HURRICANE_DATES, format="%Y-%m-%d")
# Use isin method for proper pandas comparison
df['Hurricane'] = df['Date'].isin(NY_HURRICANE_DATES).astype(int)

In [ ]:
# US Open Tennis Tournament date ranges
us_open_ranges = [
    ('2024-08-19', '2024-09-09'),
    ('2023-08-22', '2023-09-10'), 
    ('2022-08-23', '2022-09-12'),
    ('2021-08-24', '2021-09-13'),
    ('2020-08-31', '2020-09-13')
]

# Generate all dates within the US Open ranges
us_open_dates = []
for start_date, end_date in us_open_ranges:
    date_range = pd.date_range(start=start_date, end=end_date, freq='D')
    us_open_dates.extend(date_range)

US_OPEN_DATES = pd.Series(us_open_dates)
df['US_Open'] = df['Date'].isin(US_OPEN_DATES).astype(int)

**Create Lag Features** Tree models cannot handle sequences directly, so we create lagged values and rolling averages for each mode.

In [ ]:
lags = [1, 2, 3, 7] # 1-day, 2-day, 3-day, 7-day
for  col in df.columns[1:6]:
    for lag in lags:
        df[f'{col}_lag_{lag}'] = df[col].shift(lag)
    df[f'{col}_mean_7'] = df[col].shift(1).rolling(window=7).mean()

In [ ]:
df.head()

Since our dataset size is abou $1770$, we can just drop the first $7$ rows containing `NaN` values with minimal effect on training.

In [ ]:
df.dropna(inplace=True)
df.head()

In [ ]:
df.to_csv('MTA_Multimode_Ridership_Data_Processed.csv', index=False)

# 2. Preparing the Data for LightGBM Models

In [ ]:
df.columns = df.columns.str.replace('[^a-zA-Z0-9_]+', '_', regex=True)
columns = df.columns[1:]
target_columns = columns[:5]
target_columns

In [ ]:
X = df.drop(columns=['Date'] + list(target_columns))

In [ ]:
split_date = pd.to_datetime('2024-03-01')
train_mask = df['Date'] < split_date
test_mask = df['Date'] >= split_date

X_train = X[train_mask]
X_test = X[test_mask]


print(f"Training set size: {len(X_train)}")
print(f"Test set size: {len(X_test)}")
print(f"Training date range: {df[train_mask]['Date'].min()} to {df[train_mask]['Date'].max()}")
print(f"Test date range: {df[test_mask]['Date'].min()} to {df[test_mask]['Date'].max()}")

# 3. Training the LightGBM Models

In [ ]:
GBM_models = {}
MAE_results = {}

In [ ]:
for target in target_columns:
    y = df[target]
    y_train = y[train_mask]
    y_test = y[test_mask]

    model = LGBMRegressor(
        n_estimators=1000,       # more boosting rounds
        learning_rate=0.05,      # step size
        max_depth=-6,            # let model choose
        num_leaves=31,           # controls complexity (default 31)
        subsample=0.8,           # bagging
        colsample_bytree=0.8,    # feature sampling
        random_state=67
    )

    model.fit(X_train, y_train,
        eval_set=[(X_test, y_test)],
        eval_metric="mae",
        callbacks=[
            lgb.early_stopping(stopping_rounds=50),
            lgb.log_evaluation(-1)
        ],
    )

    GBM_models[target] = model
    y_pred = model.predict(X_test)
    mae = mean_absolute_error(y_test, y_pred)
    MAE_results[target] = mae
    print(f"MAE for {target}: {mae:.2f}\n")

In [ ]:
for target in target_columns:
    y_prev = df[target].shift(1).iloc[1454:]
    y_prev7 = df[target].shift(7).iloc[1454:]
    y_actual = df[target].iloc[1454:]
    mae_prev = mean_absolute_error(y_actual, y_prev)
    mae_prev7 = mean_absolute_error(y_actual, y_prev7)
    print(f"MAE for {target} (actual): {MAE_results[target]:.2f}")
    print(f"MAE for {target} (1-day lag): {mae_prev:.2f}")
    print(f"MAE for {target} (7-day lag): {mae_prev7:.2f}\n")

# 4. Understanding the Models
We use `shap` (Shapley Additive exPlanations) method to explain individual model predictions by attributing the contribution of each input feature to the outcome. We then remove features that barely contribute to improve the `MAE` of the model.

In [ ]:
import shap

#### MTA Subway SHAP

In [ ]:
subway_explainer = shap.TreeExplainer(GBM_models['Subways_Total_Estimated_Ridership'])
subway_shap_values = subway_explainer.shap_values(X_test)

plt.figure(figsize=(8, 6))  # Set smaller figure size (width, height in inches)
shap.summary_plot(subway_shap_values, X_test, show=False, max_display=10)
plt.show()

In [ ]:
subway_top_features = list(X.columns[np.argsort(-np.abs(subway_shap_values.values if hasattr(subway_shap_values, "values") else subway_shap_values).mean(0))[:7]])
subway_top_features

#### MTA Bus SHAP

In [ ]:
bus_explainer = shap.TreeExplainer(GBM_models['Buses_Total_Estimated_Ridership'])
bus_shap_values = bus_explainer.shap_values(X_test)

plt.figure(figsize=(8, 6))  # Set smaller figure size (width, height in inches)
shap.summary_plot(bus_shap_values, X_test, show=False, max_display=10)
plt.show()

In [ ]:
bus_top_features = list(X.columns[np.argsort(-np.abs(bus_shap_values.values if hasattr(bus_shap_values, "values") else bus_shap_values).mean(0))[1:8]])
# remove the first feature because of spurious correlation
bus_top_features

#### Long Island Railroad SHAP

In [ ]:
lirr_explainer = shap.TreeExplainer(GBM_models['LIRR_Total_Estimated_Ridership'])
lirr_shap_values = lirr_explainer.shap_values(X_test)

plt.figure(figsize=(8, 6))  # Set smaller figure size (width, height in inches)
shap.summary_plot(lirr_shap_values, X_test, show=False, max_display=10)
plt.show()

In [ ]:
lirr_top_features = list(X.columns[np.argsort(-np.abs(lirr_shap_values.values if hasattr(lirr_shap_values, "values") else lirr_shap_values).mean(0))[:7]])
lirr_top_features

#### Metro-North SHAP

In [ ]:
mnrr_explainer = shap.TreeExplainer(GBM_models['Metro_North_Total_Estimated_Ridership'])
mnrr_shap_values = mnrr_explainer.shap_values(X_test)

plt.figure(figsize=(8, 6))  # Set smaller figure size (width, height in inches)
shap.summary_plot(mnrr_shap_values, X_test, show=False, max_display=10)
plt.show()

In [ ]:
mnrr_top_features = list(X.columns[np.argsort(-np.abs(mnrr_shap_values.values if hasattr(mnrr_shap_values, "values") else mnrr_shap_values).mean(0))[:7]])
mnrr_top_features

#### Staten Island Railway SHAP

In [ ]:
sir_explainer = shap.TreeExplainer(GBM_models['Staten_Island_Railway_Total_Estimated_Ridership'])
sir_shap_values = sir_explainer.shap_values(X_test)

plt.figure(figsize=(8, 6))  # Set smaller figure size (width, height in inches)
shap.summary_plot(sir_shap_values, X_test, show=False, max_display=10)
plt.show()

In [ ]:
sir_top_features = list(X.columns[np.argsort(-np.abs(sir_shap_values.values if hasattr(sir_shap_values, "values") else sir_shap_values).mean(0))[:7]])
sir_top_features

# 5. Removing Extra Features and Retraining

In [ ]:
target_features = {
    'Subways_Total_Estimated_Ridership': subway_top_features,
    'Buses_Total_Estimated_Ridership': bus_top_features,
    'LIRR_Total_Estimated_Ridership': lirr_top_features,
    'Metro_North_Total_Estimated_Ridership': mnrr_top_features,
    'Staten_Island_Railway_Total_Estimated_Ridership': sir_top_features
}

In [ ]:
for target, features in target_features.items():
    modified_X = X[features]
    X_train = modified_X[train_mask]
    X_test = modified_X[test_mask]
    y = df[target]
    y_train = y[train_mask]
    y_test = y[test_mask]

    model = LGBMRegressor(
        n_estimators=1000,
        learning_rate=0.05,
        max_depth=-6,
        num_leaves=31,
        subsample=0.8,
        colsample_bytree=0.8,
        random_state=67
    )
    
    model.fit(X_train, y_train,
        eval_set=[(X_test, y_test)],
        eval_metric="mae",
        callbacks=[
            lgb.early_stopping(stopping_rounds=50),
            lgb.log_evaluation(-1)
        ],
    )

    #GBM_models[target] = model
    y_pred = model.predict(X_test)
    mae = mean_absolute_error(y_test, y_pred)
    #MAE_results[target] = mae
    print(f"MAE for {target}: {mae:.2f}\n")

# 6. Optimize Hyperparameters.
We saw that our earlier model had about the same `MAE` after removing the unnecessary features. Our next step is to tune the hyperparameters of each model.

In [130]:
import optuna
optuna.logging.set_verbosity(optuna.logging.CRITICAL)

In [134]:
def tuneModel(X_train, y_train, X_val, y_val, n_trials=50):
    def objective(trial):
        params = {
            'objective': 'regression',
            'metric': 'mae',
            'boosting_type': 'gbdt',
            'num_leaves': trial.suggest_int('num_leaves', 20, 100),
            'max_depth': trial.suggest_int('max_depth', 6, 7), # 67 haha
            'learning_rate': trial.suggest_float('learning_rate', 0.01, 0.1, log=True),
            'subsample': trial.suggest_float('subsample', 0.6, 1.0),
            'colsample_bytree': trial.suggest_float('colsample_bytree', 0.6, 1.0),
            'reg_alpha': trial.suggest_float('reg_alpha', 1e-3, 10.0, log=True),
            'reg_lambda': trial.suggest_float('reg_lambda', 1e-3, 10.0, log=True),
            'verbose': -1
        }

        temp_model = lgb.LGBMRegressor(**params, n_estimators=500, random_state=67)
        temp_model.fit(X_train, y_train,
            eval_set=[(X_val, y_val)],
            eval_metric="mae",
            callbacks=[lgb.early_stopping(stopping_rounds=50), lgb.log_evaluation(-1)],
        )
        preds = temp_model.predict(X_val)
        mae = mean_absolute_error(y_val, preds)
        return mae

    study = optuna.create_study(direction='minimize')
    study.optimize(objective, n_trials=n_trials)
    return study.best_params




In [135]:
for target, features in target_features.items():
    modified_X = X[features]
    X_train = modified_X[train_mask]
    X_test = modified_X[test_mask]
    y = df[target]
    y_train = y[train_mask]
    y_test = y[test_mask]
    best_params = tuneModel(X_train, y_train, X_test, y_test)
    print(f"Best parameters for {target}: {best_params}")

Training until validation scores don't improve for 50 rounds
Early stopping, best iteration is:
[430]	valid_0's l1: 200504
Training until validation scores don't improve for 50 rounds
Early stopping, best iteration is:
[151]	valid_0's l1: 218377
Training until validation scores don't improve for 50 rounds
Did not meet early stopping. Best iteration is:
[498]	valid_0's l1: 194194
Training until validation scores don't improve for 50 rounds
Early stopping, best iteration is:
[380]	valid_0's l1: 218962
Training until validation scores don't improve for 50 rounds
Early stopping, best iteration is:
[228]	valid_0's l1: 196806
Training until validation scores don't improve for 50 rounds
Early stopping, best iteration is:
[193]	valid_0's l1: 196403
Training until validation scores don't improve for 50 rounds
Early stopping, best iteration is:
[296]	valid_0's l1: 222492
Training until validation scores don't improve for 50 rounds
Early stopping, best iteration is:
[184]	valid_0's l1: 212582
Tra